In [1]:
%matplotlib inline
%load_ext autoreload
#%load_ext lab_black

In [2]:
from traffic.data.datasets import landing_zurich_2019 as t

/home/richard/anaconda3/envs/artefact/lib/python3.7/site-packages/pkg_resources/__init__.py:1151: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  self, resource_name
/home/richard/anaconda3/envs/artefact/lib/python3.7/site-packages/ipyleaflet/leaflet.py:317: DeprecationWarning: metadata {'dtype': None} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  data = Dataset().tag(dtype=None, sync=True, to_json=ds_x_to_json)
/home/richard/anaconda3/envs/artefact/lib/python3.7/site-packages/pyarrow/pandas_compat.py:294: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  elif isinstance(name, collections.Sequence):


In [3]:
# all flows
t_f = t.query("track == track").resample(30).unwrap("track").eval(max_workers=10)
t_f.to_parquet("data/lszh.parquet")
t_f.data.head()

,timestamp,altitude,callsign,geoaltitude,groundspeed,icao24,lastseen,latitude,longitude,onground,origin,track,vertical_rate,distance,flight_id,runway,initial_bearing,initial_flow,simple,track_unwrapped
0,2019-10-27 12:43:02+00:00,14000.0,EWG15Z,14450.0,272.677230,3c6dd4,2019-10-27 13:05:56+00:00,48.129567,8.576378,False,EDDH,164.689594,128.0,39.929720,EWG15Z_2160,14,181.562817,162-216,True,164.689594
1,2019-10-27 12:43:41+00:00,13850.0,EWG15Z,14325.0,277.483093,3c6dd4,2019-10-27 13:05:56+00:00,48.087387,8.616371,False,EDDH,144.534332,-704.0,37.480879,EWG15Z_2160,14,181.562817,162-216,True,144.534332
2,2019-10-27 12:44:21+00:00,13400.0,EWG15Z,13850.0,274.461053,3c6dd4,2019-10-27 13:05:56+00:00,48.046509,8.660042,False,EDDH,144.340984,-704.0,35.215041,EWG15Z_2160,14,181.562817,162-216,True,144.340984
3,2019-10-27 12:45:00+00:00,12925.0,EWG15Z,13375.0,277.483093,3c6dd4,2019-10-27 13:05:56+00:00,48.005645,8.703707,False,EDDH,144.534332,-832.0,33.072060,EWG15Z_2160,14,181.562817,162-216,True,144.534332
4,2019-10-27 12:45:39+00:00,12400.0,EWG15Z,12825.0,268.880782,3c6dd4,2019-10-27 13:05:56+00:00,47.965657,8.746174,False,EDDH,144.536636,-768.0,31.114796,EWG15Z_2160,14,181.562817,162-216,True,144.536636


In [4]:
stats = t.groupby("flight_id").agg({"runway": "max", "initial_flow": "max"})
stats = stats.reset_index().groupby(["runway", "initial_flow"]).count().unstack()
stats

flight_id                                 
initial_flow   162-216 24-72 240-276 312-354 90-132 N/A
runway                                                 
14                4437  2048    3437    1307   3170  58
16                   1     1       1       1      4   1
28                 899   567     678     323    668  31
34                 340   182     586     118    366   6
N/A                 76    24      91      31     22   6

In [5]:
# 1 flow
from traffic.core import Traffic
t_f = t.query("track == track and initial_flow == '162-216' and runway =='14'").resample(30).unwrap("track").eval(max_workers=10)
t_f.to_parquet("data/lszh1.parquet")
t_f.data.head()

,timestamp,altitude,callsign,geoaltitude,groundspeed,icao24,lastseen,latitude,longitude,onground,origin,track,vertical_rate,distance,flight_id,runway,initial_bearing,initial_flow,simple,track_unwrapped
0,2019-10-17 13:04:36+00:00,13400.0,EWG9776,13900.0,283.388181,3c56ed,2019-10-17 13:21:59+00:00,48.128450,8.568632,False,EDDL,191.190979,-448.0,39.855244,EWG9776_1396,14,181.161693,162-216,True,191.190979
1,2019-10-17 13:05:05+00:00,12975.0,EWG9776,13500.0,284.929570,3c56ed,2019-10-17 13:21:59+00:00,48.091782,8.560204,False,EDDL,184.630046,-1536.0,37.648575,EWG9776_1396,14,181.161693,162-216,True,184.630046
2,2019-10-17 13:05:34+00:00,12300.0,EWG9776,12800.0,282.639102,3c56ed,2019-10-17 13:21:59+00:00,48.052989,8.556589,False,EDDL,183.854528,-1408.0,35.318136,EWG9776_1396,14,181.161693,162-216,True,183.854528
3,2019-10-17 13:06:03+00:00,11575.0,EWG9776,12050.0,277.872033,3c56ed,2019-10-17 13:21:59+00:00,48.015747,8.552434,False,EDDL,184.541035,-1344.0,33.081188,EWG9776_1396,14,181.161693,162-216,True,184.541035
4,2019-10-17 13:06:32+00:00,11325.0,EWG9776,11825.0,275.655344,3c56ed,2019-10-17 13:21:59+00:00,47.978256,8.548599,False,EDDL,183.938062,-320.0,30.830059,EWG9776_1396,14,181.161693,162-216,True,183.938062


In [6]:
# 2 flows
from traffic.core import Traffic
t_f = t.query("track == track and initial_flow in ['162-216', '90-132'] and runway in ['14', '28']").unwrap("track").resample(30).eval(max_workers=10)
t_f.to_parquet("data/lszh2.parquet")
t_f.data.head()

,timestamp,altitude,callsign,geoaltitude,groundspeed,icao24,lastseen,latitude,longitude,onground,origin,track,vertical_rate,distance,flight_id,runway,initial_bearing,initial_flow,simple,track_unwrapped
0,2019-11-16 14:40:23+00:00,12875.0,SWR108Z,12775.0,305.132494,4b17df,2019-11-16 14:55:19+00:00,48.128543,8.571519,False,EDDL,181.690205,-960.0,39.863311,SWR108Z_10155,14,181.313795,162-216,True,181.690205
1,2019-11-16 14:40:47+00:00,12325.0,SWR108Z,12250.0,299.001414,4b17df,2019-11-16 14:55:19+00:00,48.095078,8.571259,False,EDDL,179.808376,-1472.0,37.854365,SWR108Z_10155,14,181.313795,162-216,True,179.808376
2,2019-11-16 14:41:12+00:00,11700.0,SWR108Z,11625.0,294.001447,4b17df,2019-11-16 14:55:19+00:00,48.060949,8.571449,False,EDDL,180.000000,-1536.0,35.806152,SWR108Z_10155,14,181.313795,162-216,True,180.000000
3,2019-11-16 14:41:36+00:00,11100.0,SWR108Z,11050.0,296.041971,4b17df,2019-11-16 14:55:19+00:00,48.028336,8.571053,False,EDDL,180.964484,-1472.0,33.848316,SWR108Z_10155,14,181.313795,162-216,True,180.964484
4,2019-11-16 14:42:01+00:00,10650.0,SWR108Z,10600.0,283.063353,4b17df,2019-11-16 14:55:19+00:00,47.994232,8.570023,False,EDDL,181.214570,-1024.0,31.800433,SWR108Z_10155,14,181.313795,162-216,True,181.214570


In [7]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from traffic.core import Traffic
from traffic.algorithms.clustering import prepare_features

list_features = ["track_unwrapped", "longitude", "latitude", "altitude"]

def to_npy(filename, reshape_type="ae"):
    t = Traffic.from_file(f"{filename}.parquet")
    nb_samples = len(t[0].data)
    nb_features = len(list_features)
    X = prepare_features(t, None, list_features)
    X = MinMaxScaler(feature_range=(-1, 1)).fit_transform(X)
    if reshape_type == "cae_keras":
        # CAE KERAS (batch_size, nb_samples, nb_features)
        X = X.reshape(-1, nb_samples, nb_features)
    elif reshape_type == "cae_torch":
        # CAE pytorch (batch_size, nb_features, nb_samples)
        X = X.reshape(-1, nb_features).T.reshape(-1, nb_features, nb_samples)
    np.save(f"{filename}_{reshape_type}.npy", X)

In [8]:
to_npy("data/lszh", "ae")

In [9]:
to_npy("data/lszh", reshape_type="cae_keras")

In [10]:
to_npy("data/lszh", reshape_type="cae_torch")